# Optimization Scenario for Manpower Planning

## Understanding Manpower Planning

Manpower planning, also known as workforce planning, is a strategic process used by organizations to ensure that they have the right number of people, with the right skills, at the right time. This process involves forecasting the organization's future human resource needs and developing strategies to meet these needs, considering both the external and internal environment in which the organization operates.

The essence of manpower planning lies in its ability to align the workforce with the organization's goals and objectives, optimizing the balance between demand for and supply of labor. This involves several key steps:

* Forecasting Labor Demand: This step involves predicting the number of employees needed in the future, based on the organization's goals, projected workloads, and market trends.

* Analyzing Current Workforce: Understanding the current workforce's composition, skills, and capabilities is important to identify gaps between current capabilities and future needs.

* Developing Employment Strategies: Based on the gap analysis, strategies are formulated to address how to source, develop, and retain the required workforce. This includes decisions on hiring, training, reassigning, or even downsizing staff.

* Implementing and Monitoring: The final step involves putting the strategies into action and continuously monitoring their effectiveness, making adjustments as necessary to ensure the organization's objectives are met efficiently.

Effective manpower planning not only helps in meeting the immediate staffing needs but also contributes to the long-term success of an organization by ensuring a consistent and capable workforce ready to tackle future challenges.

## Scenario Overview: Strategic Manpower Planning Challenge

In this project, we explore a scenario where a company is managing the complexities of manpower planning over the next 12 months. The company's objective is clear: to optimize its workforce in a way that balances the monthly labor demands with the costs associated with hiring, firing, and monthly salaries. 

### The Scenario Unfolds

* Monthly Labor Demand: The company faces varying labor demands each month, influenced by market trends, project cycles, and seasonal fluctuations.

* Hiring and Firing Costs: Bringing new employees on board or letting them go incurs significant costs, which must be carefully managed to keep expenses in check.

* Salary Obligations: Monthly salaries represent a recurring expense, directly tied to the number of employees on the payroll.

* Hiring/Firing Restrictions: Regulatory or policy constraints limit the number of employees that can be hired or fired within a given month, adding another layer of complexity to the planning process.

### Your Mission

As future HR strategists and business analysts, your task is to develop a model that minimizes total costs while ensuring the company has the manpower to meet its monthly demands.

### Mathematical Model for Manpower Planning

#### Decision Variables
Let:
- $H_t$: Number of employees hired in month $t$, $t = 1, \ldots, 12$.
- $F_t$: Number of employees fired in month $t$, $t = 1, \ldots, 12$.
- $E_t$: Number of employees at the end of month $t$, $t = 1, \ldots, 12$.

#### Parameters
- $D_t$: Demand for employees in month $t$, $t = 1, \ldots, 12$.
- $C_H$: Cost of hiring one employee.
- $C_F$: Cost of firing one employee.
- $C_S$: Monthly salary per employee.
- $MaxH$: Maximum number of employees that can be hired in a month.
- $MaxF$: Maximum number of employees that can be fired in a month.
- $E_0$: Initial number of employees.

#### Objective Function
Minimize the total cost associated with manpower planning over the planning horizon:
$$\text{Minimize } Z = \sum_{t=1}^{12} \left( C_H \cdot H_t + C_F \cdot F_t + C_S \cdot E_t \right)$$

#### Constraints
1. **Workforce Balance Constraint:**
   For each month $t$, the number of employees at the end of the month is equal to the number of employees at the end of the previous month plus the number of hires minus the number of fires.
   $$E_t = E_{t-1} + H_t - F_t, \quad \forall t = 1, \ldots, 12$$

2. **Demand Satisfaction Constraint:**
   For each month $t$, the number of employees at the end of the month must meet or exceed the demand for that month.
   $$E_t \geq D_t, \quad \forall t = 1, \ldots, 12$$

3. **Hiring Limit Constraint:**
   The number of hires in any month $t$ cannot exceed the maximum hiring limit.
   $$0 \leq H_t \leq MaxH, \quad \forall t = 1, \ldots, 12$$

4. **Firing Limit Constraint:**
   The number of fires in any month $t$ cannot exceed the maximum firing limit.
   $$0 \leq F_t \leq MaxF, \quad \forall t = 1, \ldots, 12$$

5. **Initial Condition:**
   The number of employees at the start of the planning horizon is given by $E_0$.


In [1]:
import gurobipy as gp
from gurobipy import GRB

# Parameters 
D = [100, 120, 110, 130, 140, 135, 130, 125, 120, 115, 110, 105]  # Monthly demand
C_H = 500  # Cost of hiring
C_F = 300  # Cost of firing
C_S = 2000  # Monthly salary
MaxH = 20  # Max hiring per month
MaxF = 15  # Max firing per month
E_0 = 100  # Initial employees

# Model
model = gp.Model("Manpower Planning")

# Variables
H = model.addVars(12, vtype=GRB.INTEGER, name="Hire")
F = model.addVars(12, vtype=GRB.INTEGER, name="Fire")
E = model.addVars(12, vtype=GRB.INTEGER, name="Employees")

# Objective
model.setObjective(gp.quicksum(C_H * H[t] + C_F * F[t] + C_S * E[t] for t in range(12)), GRB.MINIMIZE)

# Constraints
for t in range(12):
    if t == 0:
        model.addConstr(E[t] == E_0 + H[t] - F[t], "WorkforceBalance{}".format(t))
    else:
        model.addConstr(E[t] == E[t-1] + H[t] - F[t], "WorkforceBalance{}".format(t))
    model.addConstr(E[t] >= D[t], "Demand{}".format(t))
    model.addConstr(H[t] <= MaxH, "HiringLimit{}".format(t))
    model.addConstr(F[t] <= MaxF, "FiringLimit{}".format(t))

# Solve
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for t in range(12):
        print(f"Month {t+1}: Hire {H[t].X}, Fire {F[t].X}, Employees {E[t].X}")
else:
    print("No optimal solution found.")

# Optimization Report
model.printAttr('X')
model.printStats()


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 36 columns and 83 nonzeros
Model fingerprint: 0xddfeeed4
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]
Found heuristic solution: objective 2918500.0000
Presolve removed 40 rows and 11 columns
Presolve time: 0.00s
Presolved: 8 rows, 25 columns, 32 nonzeros
Variable types: 0 continuous, 25 integer (0 binary)

Root relaxation: cutoff, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cut